# Food Delivery Data Analysis

This notebook consolidates the entire workflow for processing and analyzing the food delivery dataset. 
It covers data loading, merging, and comprehensive analysis to answer key business questions.

In [ ]:
import pandas as pd
import re
import os

# File Paths
ORDERS_FILE = "orders.csv"
USERS_FILE = "users.json"
RESTAURANTS_FILE = "restaurants.sql"

## 1. Data Loading and Merging

In [ ]:
def load_restaurants_from_sql(file_path):
    """Parses SQL INSERT statements to create a DataFrame."""
    restaurant_data = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.strip().startswith("INSERT INTO restaurants VALUES"):
                match = re.search(r"VALUES\s*\((.*?)\);", line, re.IGNORECASE)
                if match:
                    values_str = match.group(1)
                    try:
                         # Safe evaluation of basic types (int, string, float)
                         row = eval(f"({values_str})")
                         restaurant_data.append(row)
                    except Exception:
                        pass
    return pd.DataFrame(restaurant_data, columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating'])

# Load Datasets
print("Loading datasets...")
orders_df = pd.read_csv(ORDERS_FILE)
users_df = pd.read_json(USERS_FILE)
restaurants_df = load_restaurants_from_sql(RESTAURANTS_FILE)

# Merge Data
# 1. Orders + Users (Left Join on user_id)
merged_df = pd.merge(orders_df, users_df, on='user_id', how='left')

# 2. + Restaurants (Left Join on restaurant_id)
final_df = pd.merge(merged_df, restaurants_df, on='restaurant_id', how='left', suffixes=('', '_info'))

# Convert date
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)

print(f"Final Dataset Shape: {final_df.shape}")
final_df.head()

## 2. Analysis & Answers

In [ ]:
# 1. Which city has the highest total revenue (total_amount) from Gold members?
gold_rev_city = final_df[final_df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().sort_values(ascending=False)
print(f"1. City with Highest Gold Revenue: {gold_rev_city.index[0]} (Amount: {gold_rev_city.iloc[0]:.2f})")

# 2. Highest AOV Cuisine
cuisine_aov = final_df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)
print(f"2. Highest AOV Cuisine: {cuisine_aov.index[0]} ({cuisine_aov.iloc[0]:.2f})")

# 3. Users > 1000 Total Value
user_totals = final_df.groupby('user_id')['total_amount'].sum()
print(f"3. Users > 1000: {user_totals[user_totals > 1000].count()}")

# 4. Rating Range Revenue
bins = [2.9, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']
final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels)
rating_rev = final_df.groupby('rating_range', observed=False)['total_amount'].sum().sort_values(ascending=False)
print(f"4. Top Rating Range: {rating_rev.index[0]}")

# 5. Gold City Highest AOV
gold_aov = final_df[final_df['membership'] == 'Gold'].groupby('city')['total_amount'].mean().sort_values(ascending=False)
print(f"5. Gold City Highest AOV: {gold_aov.index[0]}")

# 6. Lowest Restaurant Count Cuisine
c_stats = final_df.groupby('cuisine').agg({'restaurant_id': 'nunique', 'total_amount': 'sum'}).sort_values('restaurant_id')
print(f"6. Lowest Rest. Count Cuisine: {c_stats.index[0]}")

# 7. Gold Order %
gold_pct = (len(final_df[final_df['membership'] == 'Gold']) / len(final_df)) * 100
print(f"7. Gold Order %: {gold_pct:.0f}%")

# 8. Top Restaurant (<20 orders)
r_stats = final_df.groupby('restaurant_name').agg({'total_amount': 'mean', 'order_id': 'count'})
r_filtered = r_stats[r_stats['order_id'] < 20]
target_options = ['Grand Cafe Punjabi', 'Grand Restaurant South Indian', 'Ruchi Mess Multicuisine', 'Ruchi Foods Chinese']
matches = r_filtered[r_filtered.index.isin(target_options)].sort_values('total_amount', ascending=False)
if not matches.empty:
    print(f"8. Top Restaurant (<20 orders): {matches.index[0]} (AOV: {matches.iloc[0]['total_amount']:.2f})")
else:
    print(f"8. Top Restaurant (<20 orders) [Global]: {r_filtered.sort_values('total_amount', ascending=False).index[0]}")

# 9. Top Revenue Combo (Global vs Options)
combo = final_df.groupby(['membership', 'cuisine'])['total_amount'].sum()
print(f"9. Top Revenue Combo (Global): {combo.sort_values(ascending=False).index[0]}")
print("   Checking Options:")
opts = [('Gold', 'Indian'), ('Gold', 'Italian'), ('Regular', 'Indian'), ('Regular', 'Chinese')]
print(f"   Winner Among Options: {max(opts, key=lambda x: combo.get(x, 0))}")

# 10. Q3 Revenue
final_df['quarter'] = final_df['order_date'].dt.quarter
q_map = {1: 'Q1', 2: 'Q2', 3: 'Q3', 4: 'Q4'}
q_rev = final_df.groupby(final_df['quarter'].map(q_map))['total_amount'].sum().sort_values(ascending=False)
print(f"10. Top Quarter: {q_rev.index[0]}")

# 11. Total Gold Orders
gold_orders = len(final_df[final_df['membership'] == 'Gold'])
print(f"11. Total Gold Orders: {gold_orders}")

# 12. Hyderabad Revenue
hyd_rev = final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()
print(f"12. Hyderabad Revenue: {round(hyd_rev)}")

# 13. Distinct Users
print(f"13. Distinct Users: {final_df['user_id'].nunique()}")

# 14. Gold AOV
gold_aov_val = final_df[final_df['membership'] == 'Gold']['total_amount'].mean()
print(f"14. Gold AOV: {round(gold_aov_val, 2)}")

# 15. Ratings >= 4.5
print(f"15. Orders Rating >= 4.5: {len(final_df[final_df['rating'] >= 4.5])}")

# 16. Orders in Top Gold City
gold_df = final_df[final_df['membership'] == 'Gold']
top_gold_city = gold_df.groupby('city')['total_amount'].sum().idxmax()
orders_top_city = len(gold_df[gold_df['city'] == top_gold_city])
print(f"16. Orders in Top Gold City ({top_gold_city}): {orders_top_city}")